In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder,OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from sklearn.metrics import mean_squared_error
import time

In [ ]:
_time_cell_start = None

def tic():
    """Avvia il timer per la cella corrente."""
    global _time_cell_start
    _time_cell_start = time.time()

def toc():
    """Ferma il timer e stampa il tempo trascorso dalla chiamata a tic()."""
    if _time_cell_start is None:
        print("⚠️ Chiamare prima tic() per avviare il timer.")
        return
    elapsed = time.time() - _time_cell_start
    print(f"⏱ Tempo esecuzione cella: {elapsed:.2f} s")


# Preprocesado y limpieza de datos

Se realiza el mismo preprocesado hecho en el notebook de Sistemas Recomendadores Colaborativos y basado en contenido.

In [ ]:
# Cargamos los ficheros de nuestra base de datos
genre_columns = [
    'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
    'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
    'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
]

movies = pd.read_csv(
    '/content/drive/MyDrive/Practicas/TAAD/Movielens_100K/u.item',
    sep='|',
    encoding='latin-1',
    header=None,
    usecols=[0, 1, 2, 4] + list(range(5, 24)),
    names=['MovieID', 'Title', 'ReleaseDate', 'IMDB_URL'] + genre_columns
)


ratings = pd.read_csv('/content/drive/MyDrive/Practicas/TAAD/Movielens_100K/u.data',
                      sep='\t',
                      engine='python',
                      names=['UserID', 'MovieID', 'Rating', 'Timestamp'])



users = pd.read_csv('/content/drive/MyDrive/Practicas/TAAD/Movielens_100K/u.user',
                    sep='|',
                    engine='python',
                    names=['UserID', 'Age', 'Gender', 'Occupation', 'ZipCode'])

In [ ]:
movies = movies.drop(columns = ["IMDB_URL"])

In [ ]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [ ]:
# ------------------------------------------------------------------------------------
# 1. VERIFICACIÓN DE USUARIOS DUPLICADOS
# ------------------------------------------------------------------------------------
print("-----------------")
print("\n👥 VERIFICACIÓN DE USUARIOS")
print("-----------------")

usuarios_iniciales = len(users)

# Verificar si hay usuarios duplicados
users_duplicados = users[users.duplicated(subset=['UserID'], keep=False)]

usuarios_finales = len(users)

print(f"UserID duplicados: {len(users_duplicados)}")


-----------------

👥 VERIFICACIÓN DE USUARIOS
-----------------
UserID duplicados: 0


In [ ]:
# ------------------------------------------------------------------------------
# DETECCIÓN DE TITULOS ANOMALOS EN MOVIES
# ------------------------------------------------------------------------------
movies_iniciales = len(movies)
def detectar_titulos_anomalos(df):
    """Identifica titulos que no siguen la estructura por defecto 'Titolo (Anno)'"""
    pattern_standard = re.compile(r'^(.+)\s\((\d{4})\)$')
    anomalias = df[~df['Title'].str.match(pattern_standard)].copy()

    if not anomalias.empty:
        anomalias['Tipo_Anomalia'] = anomalias['Title'].apply(
            lambda x: 'Falta el año' if '(' not in x
            else 'Formato año errado' if not re.search(r'\(\d{4}\)', x)
            else 'Termina con un espacio' if x.endswith(' ')
            else 'Estructura atipica'
        )
        print("\n⚠️ TITULOS ANOMALOS DETECTADOS:")
        print(anomalias[['MovieID', 'Title', 'Tipo_Anomalia']].to_string(index=False))
        print("\n📊 Estadisticas anomalias:")
        print(anomalias['Tipo_Anomalia'].value_counts())
    else:
        print("\n✅ Todos los titulos siguen el formato estandard")
    return anomalias

titulos_anomalos = detectar_titulos_anomalos(movies)



⚠️ TITULOS ANOMALOS DETECTADOS:
 MovieID                                                         Title          Tipo_Anomalia
     267                                                       unknown           Falta el año
    1128                         Heidi Fleiss: Hollywood Madam (1995)  Termina con un espacio
    1201                    Marlene Dietrich: Shadow and Light (1996)  Termina con un espacio
    1412 Land Before Time III: The Time of the Great Giving (1995) (V)     Estructura atipica
    1635                                           Two Friends (1986)  Termina con un espacio

📊 Estadisticas anomalias:
Tipo_Anomalia
Termina con un espacio    3
Falta el año              1
Estructura atipica        1
Name: count, dtype: int64


In [ ]:
# ------------------------------------------------------------------------------
# LIMPIEZA DE TITULOS ANOMALOS QUE NO PERMITEN VALORAR CORRECTAMENTE EL REGISTRO
# ------------------------------------------------------------------------------

# Quitamos el registro 267 con titulo unknown y sus ratings relacionados
movies = movies[movies["MovieID"] != 267]
ratings = ratings[ratings["MovieID"] != 267]

# Quitamos (V) del registro 1412
movies.loc[movies["MovieID"] == 1412, "Title"] = "Land Before Time III: The Time of the Great Giving (1995)"

# Quitamos los espacios al final de los titulos
def clean_movie_titles(df):
    """Removes trailing spaces after the last parenthesis in movie titles."""

    def remove_trailing_space(title):
        match = re.search(r"\)\s*$", title)  # Find ')' followed by spaces at the end
        if match:
            return title[:match.start(0) + 1]  # Remove spaces, keep the ')'
        return title

    df['Title'] = df['Title'].apply(remove_trailing_space)
    return df

movies = clean_movie_titles(movies)

<ipython-input-233-1985e1495574>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Title'] = df['Title'].apply(remove_trailing_space)


In [ ]:
# ------------------------------------------------------------------------------
# DETECCIÓN DE PELÍCULAS DUPLICADAS EN MOVIES
# ------------------------------------------------------------------------------

# Verificación inicial de MovieID duplicados
print("\n🎬 VERIFICACIÓN MOVIEID DUPLICADOS")
movies_duplicados = movies[movies.duplicated(subset=['MovieID'], keep=False)]
print(f"MovieID duplicados encontrados: {len(movies_duplicados)}")

# Preparamos el DataFrame para el análisis
peliculas_limpias = movies.copy()
registros_iniciales = len(peliculas_limpias)

# Creamos columna combinada de géneros usando los nombres reales
generos_cols = [
    'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
    'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery',
    'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
]
peliculas_limpias['Genres'] = peliculas_limpias[generos_cols].apply(
    lambda row: [col for col, val in zip(generos_cols, row) if val == 1],
    axis=1
)

# Normalización de títulos (elimina año entre paréntesis)
peliculas_limpias['CleanTitle'] = (
    peliculas_limpias['Title']
    .str.extract(r'^(.*?)\s*\(\d{4}\)$')[0]
    .fillna(peliculas_limpias['Title'])
    .str.strip()
)

# Identificación de posibles duplicados
grupos_duplicados = peliculas_limpias[peliculas_limpias.duplicated('CleanTitle', keep=False)].groupby('CleanTitle')

# Proceso de eliminación de duplicados
a_eliminar = []
for titulo, grupo in grupos_duplicados:
    # Verificamos superposición de géneros
    conjuntos_generos = [set(g) for g in grupo['Genres']]
    tiene_superposicion = any(
        a & b for i, a in enumerate(conjuntos_generos)
        for j, b in enumerate(conjuntos_generos)
        if i < j
    )

    if tiene_superposicion:
        a_eliminar.extend(grupo.index.tolist()[1:])

# Eliminación de duplicados
if a_eliminar:
    peliculas_limpias = peliculas_limpias.drop(index=a_eliminar)

# Estadísticas finales
registros_finales = len(peliculas_limpias)
print("\n✅ RESULTADO FINAL:")
print(f"- Registros de películas INICIALES: {registros_iniciales}")
print(f"- Registros de películas FINALES: {registros_finales}")
print(f"- Duplicados eliminados: {registros_iniciales - registros_finales}")
print(f"- Porcentaje de duplicados: {(registros_iniciales - registros_finales)/registros_iniciales*100:.2f}%")

# Limpieza final y actualización
peliculas_limpias = peliculas_limpias.drop(columns=['CleanTitle', 'Genres'])
movies = peliculas_limpias

movies_finales = len(movies)

print("\n¡Proceso completado exitosamente!")



🎬 VERIFICACIÓN MOVIEID DUPLICADOS
MovieID duplicados encontrados: 0

✅ RESULTADO FINAL:
- Registros de películas INICIALES: 1681
- Registros de películas FINALES: 1658
- Duplicados eliminados: 23
- Porcentaje de duplicados: 1.37%

¡Proceso completado exitosamente!


In [ ]:
print("\n🔍 VERIFICANDO CONSISTENCIA...")

# Verificar que todos los MovieID en ratings existan en movies
movieids_inexistentes = ratings[~ratings['MovieID'].isin(movies['MovieID'])]
if not movieids_inexistentes.empty:
    print(f"\n🚫 CRÍTICO: Existen {len(movieids_inexistentes)} ratings con MovieID que no existen en el dataset de movies:")
    print(movieids_inexistentes['MovieID'].unique())
    print(f"\n⚙️ Eliminando los ratings con MovieID inexistentes:")
    # Eliminar ratings con MovieID inexistentes
    ratings_iniciales = len(ratings)
    ratings = ratings[ratings['MovieID'].isin(movies['MovieID'])]
    ratings_finales = len(ratings)


else:
    print("✅ Todos los MovieID en ratings existen en el catálogo de películas")

# Verificar que todos los UserID en ratings existan en users
userids_inexistentes = ratings[~ratings['UserID'].isin(users['UserID'])]
if not userids_inexistentes.empty:
    print(f"\n🚫 CRÍTICO: Existen {len(userids_inexistentes)} ratings de UserID que no existen en usuarios:")
    print(userids_inexistentes['UserID'].unique())
    print(f"\n⚙️ Eliminando los ratings con UserID inexistentes")
    # Eliminar ratings con UserID inexistentes
    ratings = ratings[ratings['UserID'].isin(users['UserID'])]
else:
    print("\n✅ Todos los UserID en ratings existen en el dataset de usuarios")

# Verificación de ratings (actualizado)
print("\n⭐ VERIFICACIÓN DE RATINGS")
rango_valido = (1, 5)
ratings_invalidos = ratings[~ratings['Rating'].between(*rango_valido)]
if ratings_invalidos.empty:
  print(f"\n✅ No hay ratings fuera de rango {rango_valido}")
else:
  print(f"\n⚠️ Hay {len(ratings_invalidos)} ratings fuera de rango {rango_valido}")
  # Eliminar ratings fuera de rango válido
  ratings = ratings[ratings['Rating'].between(*rango_valido)]

# Eliminar duplicados de ratings (mismo UserID + MovieID)
ratings = ratings.drop_duplicates(subset=['UserID', 'MovieID'], keep='first')


🔍 VERIFICANDO CONSISTENCIA...

🚫 CRÍTICO: Existen 796 ratings con MovieID que no existen en el dataset de movies:
[ 486 1444  268  673  303  680 1003 1286  670  500  348  865  881 1257
 1617 1607 1654 1650 1658 1606 1542 1680 1625]

⚙️ Eliminando los ratings con MovieID inexistentes:

✅ Todos los UserID en ratings existen en el dataset de usuarios

⭐ VERIFICACIÓN DE RATINGS

✅ No hay ratings fuera de rango (1, 5)


In [ ]:
# ------------------------------------------------------------------------------
# REPORTE FINAL
# ------------------------------------------------------------------------------

print("\n📊 RESUMEN DE CAMBIOS:")
print(f"- Ratings: {ratings_iniciales} → {ratings_finales} (eliminados {ratings_iniciales-ratings_finales})")

print(f"- Películas: {movies_iniciales} → {movies_finales} (eliminados {movies_iniciales-movies_finales})")

print(f"- Usuarios: {usuarios_iniciales} → {usuarios_finales} (eliminados {usuarios_iniciales-usuarios_finales})")

print("\n🔍 DATOS ELIMINADOS:")
print(f"- Ratings con MovieID inválidos: {len(ratings) - len(ratings[ratings['MovieID'].isin(movies['MovieID'])])}")
print(f"- Ratings con UserID inválidos: {len(ratings) - len(ratings[ratings['UserID'].isin(users['UserID'])])}")
print(f"- Ratings fuera de rango: {len(ratings_invalidos)}")
print(f"- Ratings duplicados: {len(ratings) - len(ratings.drop_duplicates(subset=['UserID', 'MovieID']))}")

# Guardar datos limpios (opcional)
# ratings_clean.to_csv('ratings_clean.csv', index=False)
# movies_clean.to_csv('movies_clean.csv', index=False)
# users_clean.to_csv('users_clean.csv', index=False)

print("\n✅ LIMPIEZA COMPLETADA")


📊 RESUMEN DE CAMBIOS:
- Ratings: 99991 → 99195 (eliminados 796)
- Películas: 1682 → 1658 (eliminados 24)
- Usuarios: 943 → 943 (eliminados 0)

🔍 DATOS ELIMINADOS:
- Ratings con MovieID inválidos: 0
- Ratings con UserID inválidos: 0
- Ratings fuera de rango: 0
- Ratings duplicados: 0

✅ LIMPIEZA COMPLETADA


In [ ]:
movies['Title'] = movies['Title'].str.replace(r'\(\d{4}\)', '', regex=True).str.strip()

In [ ]:
# identifica dinámicamente todas las columnas de géneros 0/1 (suponiendo que MovieID, Title y ReleaseDate sean las tres primeras)
genre_columns = movies.columns[3:]

# crea la columna 'Genres' con los nombres de los géneros cuyo valor es 1
movies['Genres'] = (
     movies[genre_columns]
     .apply(lambda row: ' '.join(col for col, flag in row.items() if flag == 1), axis=1)
)


movies['ReleaseDate'] = pd.to_datetime(movies['ReleaseDate'])


movies.head()

,MovieID,Title,ReleaseDate,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Genres
0,1,Toy Story,1995-01-01,0,0,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,Animation Children's Comedy
1,2,GoldenEye,1995-01-01,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Action Adventure Thriller
2,3,Four Rooms,1995-01-01,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller
3,4,Get Shorty,1995-01-01,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,Action Comedy Drama
4,5,Copycat,1995-01-01,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,Crime Drama Thriller


Para realizar este sistema recomendador de tipo hibrido, intenté **comparar dos enfoques diferentes**, el primero sin enriquecer los embeddings, el segundo que tenía en cuenta también la información sobre usuarios y películas (dataset movies y users).

# Modelo 1: DeepMF sin enriquecimiento de embeddings y AutoEncoder

Para este modelo se han considerado únicamente los identificadores de usuario e ítem como entrada, aprendiendo embeddings básicos sin información adicional

Etapas del código:

1. **Preparación:** codificación de IDs, división train/test y construcción de matriz de ratings.

2. **Modelos:** se definen DeepMF y AutoEncoder como predictores de ratings.

3. **Validación cruzada:** se optimiza alpha, el peso ideal para combinar ambos modelos.

4. **Entrenamiento final:** se entrena con todos los datos de entrenamiento.

5. **Evaluación:** se combinan predicciones y se calcula el RMSE/MAE final en test.

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# ============================================================
# 0. Librerías y preparación inicial de datos
# ============================================================

import numpy as np, pandas as pd, tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

# ---------- Codificación de usuarios y películas ----------
# Convertimos IDs categóricos a numéricos consecutivos para que los modelos puedan procesarlos
u_enc, m_enc = LabelEncoder(), LabelEncoder()
ratings["UserID"] = u_enc.fit_transform(ratings["UserID"])  # Ej: usuario "A123" → 0, "B456" → 1
ratings["MovieID"] = m_enc.fit_transform(ratings["MovieID"])  # Ej: película "X789" → 0, "Y012" → 1

# Contamos usuarios y películas únicos
n_users = ratings["UserID"].nunique()
n_items = ratings["MovieID"].nunique()

# ---------- División train/test ----------
# Separamos el 20% de datos para prueba, manteniendo proporción de usuarios (stratify)
train_df, test_df = train_test_split(ratings, test_size=0.2,
                                    random_state=42, stratify=ratings['UserID'])

# ---------- Matriz completa de ratings ----------
# Creamos matriz usuarios×películas (faltantes se rellenan con 0)
R_full = (pd.pivot_table(ratings, values="Rating",
                        index="UserID", columns="MovieID")
         .fillna(0).values)  # Formato numpy array (n_users, n_items)

# ============================================================
# 1. Definición de modelos
# ============================================================

def build_deepmf_model(n_users: int, n_items: int,
                      emb_dim: int = 64) -> tf.keras.Model:
    """Construye modelo Deep Matrix Factorization (embedding + MLP)"""
    # Capas de entrada
    u_in = tf.keras.Input((1,), name="user")
    i_in = tf.keras.Input((1,), name="item")

    # Embeddings (representaciones densas de usuarios/películas)
    u_emb = tf.keras.layers.Embedding(n_users, emb_dim, name="u_emb")(u_in)
    i_emb = tf.keras.layers.Embedding(n_items, emb_dim, name="i_emb")(i_in)

    # Concatenamos embeddings y pasamos por MLP
    x = tf.keras.layers.Concatenate()(
        [tf.keras.layers.Flatten()(u_emb), tf.keras.layers.Flatten()(i_emb)])
    x = tf.keras.layers.Dense(128, activation="relu")(x)  # Capa oculta 1
    x = tf.keras.layers.Dropout(0.3)(x)  # Regularización para evitar overfitting
    x = tf.keras.layers.Dense(64, activation="relu")(x)   # Capa oculta 2
    x = tf.keras.layers.Dropout(0.3)(x)
    out = tf.keras.layers.Dense(1, activation="linear")(x)  # Salida: predicción de rating

    model = tf.keras.Model([u_in, i_in], out)
    model.compile(optimizer="adam", loss="mse")  # Optimizador Adam, pérdida MSE
    return model

def build_autoencoder(n_items: int, enc_dim: int = 128) -> tf.keras.Model:
    """Construye autoencoder para aprender representaciones comprimidas de usuarios"""
    inp = tf.keras.Input((n_items,))  # Entrada: vector de ratings de un usuario

    # Encoder: reduce dimensionalidad
    enc = tf.keras.layers.Dense(enc_dim, activation="relu")(inp)

    # Decoder: reconstruye el input original
    dec = tf.keras.layers.Dense(n_items, activation="linear")(enc)

    ae = tf.keras.Model(inp, dec)
    ae.compile(optimizer="adam", loss="mse")
    return ae

# ============================================================
# 2. Validación cruzada para optimizar alpha (mezcla de modelos)
# ============================================================

# Hiperparámetros
KFOLDS, EPOCHS_DMF, EPOCHS_AE = 5, 20, 20  # 5 folds, 20 épocas máximo por modelo
BATCH_DMF, BATCH_AE = 512, 256  # Tamaño de lotes
alphas = np.linspace(0, 1, 21)  # Valores de alpha a probar (0=100% AE, 1=100% DeepMF)
rmse_accum = np.zeros_like(alphas)  # Acumula RMSE para cada alpha

kf = KFold(n_splits=KFOLDS, shuffle=True, random_state=42)  # Configuración KFold

for fold, (tr_idx, va_idx) in enumerate(kf.split(train_df), 1):
    print(f"🌀 Fold {fold}/{KFOLDS}")
    tr, va = train_df.iloc[tr_idx], train_df.iloc[va_idx]  # Split train/validation

    # ----- Entrenamiento DeepMF -----
    dmf = build_deepmf_model(n_users, n_items)
    dmf.fit([tr["UserID"], tr["MovieID"]], tr["Rating"],
            epochs=EPOCHS_DMF, batch_size=BATCH_DMF,
            validation_split=0.1, verbose=0,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)])

    # Predicciones DeepMF en validation
    u_va = np.array(va["UserID"]).reshape(-1,1)
    i_va = np.array(va["MovieID"]).reshape(-1,1)
    y_va_mf = dmf.predict([u_va, i_va], batch_size=BATCH_DMF, verbose=0).flatten()

    # ----- Entrenamiento Autoencoder (solo con usuarios de train) -----
    R_train = R_full.copy()
    mask_users = tr["UserID"].unique()  # Solo usuarios en training fold
    ae = build_autoencoder(n_items)
    ae.fit(R_train[mask_users], R_train[mask_users],
           epochs=EPOCHS_AE, batch_size=BATCH_AE,
           validation_split=0.1, verbose=0,
           callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)])

    # Predicciones AE en validation (para todos los usuarios)
    R_hat = ae.predict(R_full, batch_size=BATCH_AE, verbose=0)
    y_va_ae = R_hat[u_va.flatten(), i_va.flatten()]

    # ----- Evaluación de diferentes alphas -----
    y_va_true = va["Rating"].values
    for j, a in enumerate(alphas):
        blend = a * y_va_mf + (1-a) * y_va_ae  # Mezcla lineal
        rmse = np.sqrt(mean_squared_error(y_va_true, blend))
        rmse_accum[j] += rmse  # Acumulamos RMSE para este alpha

# Promediamos RMSE entre folds
rmse_accum /= KFOLDS
best_alpha = alphas[rmse_accum.argmin()]  # Alpha con menor RMSE
print(f"\n🎯 α óptimo = {best_alpha:.2f} | RMSE medio CV = {rmse_accum.min():.4f}")

# ============================================================
# 3. Entrenamiento final y evaluación
# ============================================================

print("\n🏁 Entrenamiento final con todo el train_full...")

# ----- DeepMF final -----
tic()
deepmf_final = build_deepmf_model(n_users, n_items)
deepmf_final.fit([train_df["UserID"], train_df["MovieID"]], train_df["Rating"],
                 epochs=EPOCHS_DMF, batch_size=BATCH_DMF, verbose=0,
                 validation_split=0.1,
                 callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)])

# ----- Autoencoder final -----
ae_final = build_autoencoder(n_items)
ae_final.fit(R_full, R_full, epochs=EPOCHS_AE, batch_size=BATCH_AE,
             validation_split=0.1, verbose=0,
             callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)])
toc()
# ----- Ensamblado y evaluación en test -----
u_test = np.array(test_df["UserID"]).reshape(-1,1)
i_test = np.array(test_df["MovieID"]).reshape(-1,1)

# Predicciones DeepMF
y_test_mf = deepmf_final.predict([u_test, i_test], batch_size=BATCH_DMF, verbose=0).flatten()

# Predicciones Autoencoder
R_hat_full = ae_final.predict(R_full, batch_size=BATCH_AE, verbose=0)
y_test_ae = R_hat_full[u_test.flatten(), i_test.flatten()]

# Mezcla óptima según alpha encontrado en CV
y_blend = best_alpha * y_test_mf + (1-best_alpha) * y_test_ae

# Cálculo de RMSE final
rmse_final = np.sqrt(mean_squared_error(test_df["Rating"].values, y_blend))
mae_final = mean_absolute_error(test_df["Rating"].values, y_blend)

print(f"\n🚀 RMSE del ensemble (α={best_alpha:.2f}) en TEST: {rmse_final:.4f}")
print(f"🚀 MAE del ensemble (α={best_alpha:.2f}) en TEST: {mae_final:.4f}")

🌀 Fold 1/5
🌀 Fold 2/5
🌀 Fold 3/5
🌀 Fold 4/5
🌀 Fold 5/5

🎯 α óptimo = 0.95 | RMSE medio CV = 0.9415

🏁 Entrenamiento final con todo el train_full...
⏱ Tempo esecuzione cella: 26.12 s

🚀 RMSE del ensemble (α=0.95) en TEST: 0.9354
🚀 MAE del ensemble (α=0.95) en TEST: 0.7465


# Modelo 2: DeepMF con enriquecimiento de embeddings y AutoEncoder

**Incorpora características adicionales de usuarios** (edad, género, ocupación) y películas (género, año de estreno) junto con los embeddings. Esto permite al modelo aprender representaciones más ricas y personalizadas.

El Autoencoder usa la misma matriz de ratings, pero el sistema general debería beneficiarse del contexto añadido.

**Etapas del código:**

1.   **Preparación de datos:** codificación, normalización y creación de features para usuarios y películas.
2.   **Entrenamiento:** se ajustan dos modelos (DeepMF y AutoEncoder) con validación cruzada.
3. **Optimización:** búsqueda del mejor alpha para combinar ambos modelos.
4. **Evaluación final:** se entrena con todo el set y se evalúa con mezcla adaptativa cold vs warm users.
5. **Métricas:** se reportan RMSE y MAE para total, usuarios cold y warm.

In [ ]:
"""
Sistema de Recomendación Híbrido: DeepMF + AutoEncoder
-----------------------------------------------------
Este sistema combina:
1. Deep Matrix Factorization (DeepMF) - Para capturar patrones complejos usuario-ítem
2. AutoEncoder - Para aprender representaciones densas de las preferencias de usuarios
3. Mezcla adaptativa - Combina ambas predicciones con pesos distintos para usuarios nuevos (cold) y existentes (warm)
"""

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MultiLabelBinarizer

# ------------------------------------------------
# 0. Preparación de Datos
# ------------------------------------------------

# 0.1 Codificación de IDs únicos
# --------------------------------------------------
# Combinamos todos los IDs de usuarios y películas para evitar errores de labels no vistos
all_user_ids = pd.concat([ratings['UserID'], users['UserID']]).unique()
all_movie_ids = pd.concat([ratings['MovieID'], movies['MovieID']]).unique()

# Creamos codificadores numéricos para usuarios y películas
u_enc = LabelEncoder().fit(all_user_ids)
m_enc = LabelEncoder().fit(all_movie_ids)

# Aplicamos la codificación a los datos
ratings['UserID_enc'] = u_enc.transform(ratings['UserID'])
ratings['MovieID_enc'] = m_enc.transform(ratings['MovieID'])
movies['MovieID_enc'] = m_enc.transform(movies['MovieID'])

# Obtenemos el número total de usuarios y películas únicas
n_users = len(u_enc.classes_)
n_items = len(m_enc.classes_)

# 0.2 Características de Usuarios
# --------------------------------------------------
# Preparamos características demográficas de usuarios:
# - Edad escalada
# - Género one-hot encoded
# - Ocupación one-hot encoded

users['UserID_enc'] = u_enc.transform(users['UserID'])
users = users.set_index('UserID_enc')  # Usamos ID codificado como índice

# Escalamos la edad
scaler_age = StandardScaler()
age_scaled = scaler_age.fit_transform(users[['Age']])

# Codificación one-hot para género y ocupación
ohe_gen = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
ohe_occ = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
gen_ohe = ohe_gen.fit_transform(users[['Gender']])
occ_ohe = ohe_occ.fit_transform(users[['Occupation']])

# Combinamos todas las características
user_features = np.hstack([age_scaled, gen_ohe, occ_ohe])

# Creamos DataFrame con nombres descriptivos de columnas
user_features_df = pd.DataFrame(
    user_features,
    index=users.index,
    columns=(['Age_scaled'] +
             [f'Gen_{g}' for g in ohe_gen.categories_[0]] +
             [f'Occ_{o}' for o in ohe_occ.categories_[0]])
)

# Aseguramos que tengamos exactamente n_users filas
user_features_df = user_features_df.reindex(range(n_users), fill_value=0)

# 0.3 Características de Películas
# --------------------------------------------------
# Preparamos características de películas:
# - Géneros (multi-label one-hot encoded)
# - Año de lanzamiento escalado

movies = movies.set_index('MovieID_enc')

# Separamos los géneros (pueden ser múltiples por película)
movies['GenreList'] = movies['Genres'].str.split(r'[|\\s]+')

# Obtenemos todos los géneros únicos
all_genres = sorted({g for sub in movies['GenreList'] for g in sub})

# Codificación multi-label one-hot para géneros
mlb = MultiLabelBinarizer()
genre_ohe = mlb.fit_transform(movies['GenreList'])

# Creamos DataFrame de géneros
genre_cols = [f'genre_{g}' for g in mlb.classes_]
genre_df = pd.DataFrame(genre_ohe, columns=genre_cols, index=movies.index)

# Unimos con el DataFrame original
movies = pd.concat([movies, genre_df], axis=1)

# Procesamos el año de lanzamiento
# movies['ReleaseDate'] = pd.to_datetime(movies['ReleaseDate'], errors='coerce')
movies['ReleaseYear'] = movies['ReleaseDate'].dt.year
# movies['ReleaseYear'] = movies['ReleaseYear'].fillna(movies['ReleaseYear'].median())  # Imputamos valores faltantes
movies['ReleaseYear'] = movies['ReleaseYear'].astype(int)

# Escalamos el año
scaler_year = StandardScaler()
movies['Year_scaled'] = scaler_year.fit_transform(movies[['ReleaseYear']])

# Combinamos características de películas
movie_features = np.hstack([
    movies[genre_cols].values,
    movies[['Year_scaled']].values
])

# Creamos DataFrame final
movie_features_df = pd.DataFrame(
    movie_features,
    index=movies.index,
    columns=genre_cols + ['Year_scaled']
).fillna(0)  # Rellenamos cualquier NaN restante

# 0.4 Matriz de Ratings para el AutoEncoder
# --------------------------------------------------
# Creamos matriz usuario × película con ratings
pivot = ratings.pivot_table(values='Rating', index='UserID_enc', columns='MovieID_enc')

# Aseguramos dimensión (n_users × n_items)
pivot = pivot.reindex(index=range(n_users), columns=range(n_items))

# Estrategia de imputación en dos niveles:
# 1. Primero llenamos con la media del usuario
user_means = pivot.mean(axis=1)
pivot = pivot.apply(lambda r: r.fillna(user_means[r.name]), axis=1)

# 2. Luego con la media global para cualquier NaN restante
pivot = pivot.fillna(pivot.mean().mean())

R_full = pivot.values  # Matriz completa
R_full_norm = (R_full - 1) / 4  # Escalamos ratings [1,5] → [0,1]

# 0.5 Pesos de Muestreo y Mascara Cold-Start
# --------------------------------------------------
# Usuarios con pocas valoraciones (cold-start) necesitan tratamiento especial
user_cnt = ratings.groupby('UserID_enc')['Rating'].count()
COLD_TH = 50  # Umbral para considerar cold-start
cold_mask = user_cnt <= COLD_TH

# Pesos para balancear la influencia de usuarios con muchas/menos valoraciones
weights = (1 / user_cnt.pow(0.3)).values  # Penalizamos usuarios con muchas valoraciones
weights *= len(ratings) / weights.sum()  # Normalizamos
ratings['sample_w'] = ratings['UserID_enc'].map(dict(zip(user_cnt.index, weights)))

# 0.6 División Train/Test
# --------------------------------------------------
# Estratificamos por usuario para mantener proporción en ambos conjuntos
train_df, test_df = train_test_split(
    ratings, test_size=0.2, random_state=42, stratify=ratings['UserID_enc'])

# ------------------------------------------------
# 1. Definición de Modelos
# ------------------------------------------------

def build_deepmf_with_features(n_users, n_items, n_user_feats, n_item_feats, emb_dim=128):
    """Construye modelo DeepMF con características adicionales"""
    # Capas de entrada
    u_in = tf.keras.Input((1,), name='user_id')
    i_in = tf.keras.Input((1,), name='item_id')
    u_feat_in = tf.keras.Input((n_user_feats,), name='user_feats')
    i_feat_in = tf.keras.Input((n_item_feats,), name='item_feats')

    # Embeddings para usuarios y películas
    u_emb = tf.keras.layers.Flatten()(tf.keras.layers.Embedding(n_users, emb_dim)(u_in))
    i_emb = tf.keras.layers.Flatten()(tf.keras.layers.Embedding(n_items, emb_dim)(i_in))

    # Concatenamos embeddings con características adicionales
    x = tf.keras.layers.Concatenate()([u_emb, i_emb, u_feat_in, i_feat_in])

    # Capas ocultas
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)  # Regularización
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    # Capa de salida (predicción de rating)
    out = tf.keras.layers.Dense(1)(x)

    model = tf.keras.Model([u_in, i_in, u_feat_in, i_feat_in], out)
    model.compile(optimizer='adam', loss='mse')
    return model

def build_autoencoder(n_items, enc_dim=128):
    """Construye autoencoder para aprender representaciones densas"""
    inp = tf.keras.Input((n_items,))

    # Encoder
    enc = tf.keras.layers.Dense(enc_dim, activation='relu')(inp)

    # Decoder (reconstruye input original)
    dec = tf.keras.layers.Dense(n_items, activation='sigmoid')(enc)

    ae = tf.keras.Model(inp, dec)
    ae.compile(optimizer='adam', loss='mse')
    return ae

# ------------------------------------------------
# 2. Ajuste de Hiperparámetros (alpha)
# ------------------------------------------------
# Encontrar el mejor peso para mezclar DeepMF y AutoEncoder

KFOLDS, EPOCHS, BATCH = 5, 20, 512  # Configuración
grid = np.linspace(0,1,21)  # Valores de alpha a probar (0 a 1)
rmse_all = np.zeros_like(grid)  # Para almacenar resultados
rmse_cold = np.zeros_like(grid)
kf = KFold(n_splits=KFOLDS, shuffle=True, random_state=42)

for tr_idx, va_idx in kf.split(train_df):
    tr, va = train_df.iloc[tr_idx], train_df.iloc[va_idx]

    # Preparamos características
    tr_u_feats = user_features_df.loc[tr['UserID_enc']].values
    tr_i_feats = movie_features_df.loc[tr['MovieID_enc']].values
    va_u_feats = user_features_df.loc[va['UserID_enc']].values
    va_i_feats = movie_features_df.loc[va['MovieID_enc']].values

    # Entrenamos DeepMF
    dmf = build_deepmf_with_features(
        n_users, n_items,
        user_features_df.shape[1],
        movie_features_df.shape[1]
    )
    dmf.fit(
        [tr['UserID_enc'], tr['MovieID_enc'], tr_u_feats, tr_i_feats],
        tr['Rating'],
        sample_weight=tr['sample_w'],  # Usamos pesos para balancear
        epochs=EPOCHS,
        batch_size=BATCH,
        validation_split=0.1,
        verbose=0,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True)]
    )

    # Entrenamos AutoEncoder solo con usuarios de entrenamiento
    mask_u = tr['UserID_enc'].unique()
    ae = build_autoencoder(n_items)
    ae.fit(
        R_full_norm[mask_u], R_full_norm[mask_u],  # Solo usuarios de train
        epochs=EPOCHS,
        batch_size=BATCH//2,
        verbose=0,
        validation_split=0.1,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)]
    )

    # Predicciones
    u_va = va['UserID_enc'].to_numpy()
    i_va = va['MovieID_enc'].to_numpy()

    y_mf = dmf.predict(
        [u_va.reshape(-1,1), i_va.reshape(-1,1), va_u_feats, va_i_feats],
        batch_size=BATCH,
        verbose=0
    ).flatten()

    y_ae = ae.predict(R_full_norm, batch_size=BATCH//2, verbose=0)[u_va, i_va]*4+1  # Escalamos de vuelta [0,1]→[1,5]

    # Calculamos RMSE para diferentes valores de alpha
    y_true = va['Rating'].to_numpy()
    cold_va = cold_mask[u_va].to_numpy()

    for j, a in enumerate(grid):
        blend = a*y_mf + (1-a)*y_ae  # Mezcla lineal
        rmse_all[j] += np.sqrt(mean_squared_error(y_true, blend))
        if cold_va.any():
            rmse_cold[j] += np.sqrt(mean_squared_error(y_true[cold_va], blend[cold_va]))

# Promediamos resultados de los folds
rmse_all /= KFOLDS
rmse_cold /= KFOLDS

# Encontramos los mejores alpha
alpha_best = grid[rmse_all.argmin()]  # Para usuarios warm
alpha_best_cold = grid[rmse_cold.argmin()]  # Para usuarios cold
print(f"alpha_warm/hot={alpha_best:.2f}, alpha_cold={alpha_best_cold:.2f}")

# ------------------------------------------------
# 3. Entrenamiento Final y Evaluación
# ------------------------------------------------

# Modelos finales
deepmf_final = build_deepmf_with_features(
    n_users, n_items,
    user_features_df.shape[1],
    movie_features_df.shape[1]
)

# Preparamos características
train_u_feats = user_features_df.loc[train_df['UserID_enc']].values
train_i_feats = movie_features_df.loc[train_df['MovieID_enc']].values
test_u_feats = user_features_df.loc[test_df['UserID_enc']].values
test_i_feats = movie_features_df.loc[test_df['MovieID_enc']].values

# Entrenamiento DeepMF
tic()
deepmf_final.fit(
    [train_df['UserID_enc'], train_df['MovieID_enc'], train_u_feats, train_i_feats],
    train_df['Rating'],
    sample_weight=train_df['sample_w'],
    epochs=EPOCHS,
    batch_size=BATCH,
    validation_split=0.1,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)],
    verbose=0
)

# Entrenamiento AutoEncoder (todos los usuarios)
ae_final = build_autoencoder(n_items)
ae_final.fit(
    R_full_norm, R_full_norm,
    epochs=EPOCHS,
    batch_size=BATCH//2,
    validation_split=0.1,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)],
    verbose=0
)
toc()
# Fase de prueba
u_t = test_df['UserID_enc'].to_numpy()
i_t = test_df['MovieID_enc'].to_numpy()

# Predicciones
y_mf_test = deepmf_final.predict(
    [u_t.reshape(-1,1), i_t.reshape(-1,1), test_u_feats, test_i_feats],
    batch_size=BATCH
).flatten()

y_ae_test = ae_final.predict(R_full_norm, batch_size=BATCH//2, verbose=0)[u_t, i_t]*4+1

# Mezcla adaptativa:
# - Usamos alpha_best_cold para usuarios cold-start
# - alpha_best para los demás
alpha_vec = np.where(cold_mask[u_t], alpha_best_cold, alpha_best)
y_blend = alpha_vec*y_mf_test + (1-alpha_vec)*y_ae_test

# Cálculo de métricas
nmask_cold = cold_mask[u_t].to_numpy()
rmse_total = np.sqrt(mean_squared_error(test_df['Rating'].to_numpy(), y_blend))
mae_total = mean_absolute_error(test_df['Rating'].to_numpy(), y_blend)
rmse_cold_test = np.sqrt(mean_squared_error(
    test_df['Rating'].to_numpy()[nmask_cold],
    y_blend[nmask_cold]
))
mae_cold_test = mean_absolute_error(
    test_df['Rating'].to_numpy()[nmask_cold],
    y_blend[nmask_cold]
)
rmse_warm_test = np.sqrt(mean_squared_error(
    test_df['Rating'].to_numpy()[~nmask_cold],
    y_blend[~nmask_cold]
))
mae_warm_test = mean_absolute_error(
    test_df['Rating'].to_numpy()[~nmask_cold],
    y_blend[~nmask_cold]
)

# Resultados finales
print(f"🚀 RMSE total: {rmse_total:.4f}")
print(f"🚀 MAE total: {mae_total:.4f}")
print(f"🔍 RMSE cold (<= {COLD_TH} ratings): {rmse_cold_test:.4f}")
print(f"🔍 MAE cold (<= {COLD_TH} ratings): {mae_cold_test:.4f}")
print(f"🔍 RMSE warm (> {COLD_TH} ratings): {rmse_warm_test:.4f}")
print(f"🔍 MAE warm (> {COLD_TH} ratings): {mae_warm_test:.4f}")

alpha_warm/hot=0.90, alpha_cold=0.75
⏱ Tempo esecuzione cella: 44.56 s
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
🚀 RMSE total: 0.9246
🚀 MAE total: 0.7303
🔍 RMSE cold (<= 50 ratings): 1.0251
🔍 MAE cold (<= 50 ratings): 0.8183
🔍 RMSE warm (> 50 ratings): 0.9102
🔍 MAE warm (> 50 ratings): 0.7185


# Recomendaciones intra-set


Para poder evaluar las recomendaciones, las hacemos solo entre las película para las que tenemos una valoración del usuario.

In [ ]:
def recommend_hybrid_seen_only(user_id_original, k=10, threshold=3.5, df_source=None):
    """
    Genera recomendaciones SOLO sobre películas ya vistas por el usuario.
    Devuelve un DataFrame con el título, puntuación predicha (redondeada),
    puntuación real y si fue relevante según el umbral.
    """

    # Verifica si el usuario existe en el sistema
    if user_id_original not in u_enc.classes_:
        raise ValueError("Usuario no reconocido en el sistema.")

    # Usa todo el dataset si no se especifica otro DataFrame (como test_df)
    if df_source is None:
        df_source = ratings

    # Codifica el ID del usuario a su versión numérica interna
    user_id = u_enc.transform([user_id_original])[0]

    # Extrae las películas que el usuario ya ha visto
    user_seen_df = df_source[df_source['UserID_enc'] == user_id]
    seen_movie_ids = user_seen_df['MovieID_enc'].values

    # Prepara entradas para DeepMF y AutoEncoder
    u_ids = np.full((len(seen_movie_ids), 1), user_id)  # IDs del usuario repetidos
    i_ids = seen_movie_ids.reshape(-1, 1)               # IDs de películas vistas
    u_feats = np.repeat(user_features_df.loc[[user_id]].values, len(seen_movie_ids), axis=0)
    i_feats = movie_features_df.loc[seen_movie_ids].values

    # Predicciones de ambos modelos
    y_mf = deepmf_final.predict([u_ids, i_ids, u_feats, i_feats], batch_size=256, verbose=0).flatten()
    y_ae = ae_final.predict(R_full_norm[np.newaxis, user_id], batch_size=1, verbose=0)[0, seen_movie_ids] * 4 + 1

    # Mezcla adaptativa con alpha distinto según si el usuario es cold-start o no
    alpha = alpha_best_cold if cold_mask[user_id] else alpha_best
    y_blend = alpha * y_mf + (1 - alpha) * y_ae

    # Selecciona las k películas con mayor puntuación predicha
    top_indices = np.argsort(y_blend)[::-1][:k]
    top_movie_ids_enc = seen_movie_ids[top_indices]
    top_scores = round_down_half(y_blend[top_indices])  # Redondea al 0.5 inferior

    # Recupera información adicional
    movie_ids_original = m_enc.inverse_transform(top_movie_ids_enc)
    titles = movies.loc[top_movie_ids_enc, 'Title'].values
    rating_real = user_seen_df.set_index('MovieID_enc').loc[top_movie_ids_enc, 'Rating'].values

    # Determina si el rating real supera el umbral (película relevante o no)
    relevant_flags = rating_real >= threshold

    # Construye el DataFrame final con resultados
    results = pd.DataFrame({
        'MovieID': movie_ids_original,
        'Title': titles,
        'RatingPred': top_scores,
        'RatingReal': rating_real,
        'Relevant': relevant_flags
    })

    # Devuelve la tabla con comparaciones
    return results


In [ ]:
recommend_hybrid_seen_only(1, k=10, threshold=3.5, df_source=test_df)


,MovieID,Title,RatingPred,RatingReal,Relevant
0,169,"Wrong Trousers, The",5.0,5,True
1,183,Alien,4.5,5,True
2,48,Hoop Dreams,4.5,5,True
3,14,"Postino, Il",4.5,5,True
4,23,Taxi Driver,4.5,4,True
5,100,Fargo,4.5,5,True
6,221,Breaking the Waves,4.0,5,True
7,61,Three Colors: White,4.0,4,True
8,180,Apocalypse Now,4.0,3,False
9,173,"Princess Bride, The",4.0,5,True
